# LightGBM


# 데이터 준비 및 전처리 
- combined_test_data_200.csv (test_data + 기타소음4)


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from lightgbm import LGBMClassifier
import librosa
import soundfile as sf
import os
from tqdm import tqdm

In [2]:
# 학습 데이터 준비
df = pd.read_csv("combined_test_data_200.csv", encoding='utf-8')
feature_cols = [f'mfcc_{i}' for i in range(1, 51)]
df = df[feature_cols + ['category_03']]

# 소음 카테고리 변환
def categorize_noise(category):
    if category in ['이륜차경적']:
        return '이륜차경적'
    elif category in ['이륜차주행음']:
        return '이륜차주행음'
    elif category in ['차량사이렌']:
        return '차량사이렌'
    elif category in ['차량주행음']:
        return '차량주행음'
    elif category in ['차량경적']:
        return '차량경적'
    else:
        return '기타소음'

df['label'] = df['category_03'].apply(categorize_noise)
df = df.drop('category_03', axis=1)

X = df[feature_cols]
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) # stratify=y: 각 소음 카테고리의 비율을 유지하면서 분할

#데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#레이블 인코딩 (문자 → 숫자)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# 모델 학습 
model = LGBMClassifier(n_estimators=200, random_state=42, max_depth=15)
model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred = model.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9281914893617021
Classification Report:
               precision    recall  f1-score   support

        기타소음       0.94      0.99      0.97       373
       이륜차경적       0.96      0.90      0.93       104
      이륜차주행음       0.88      0.81      0.84       100
        차량경적       0.91      0.89      0.90        80
       차량사이렌       0.96      0.88      0.92        50
       차량주행음       0.84      0.82      0.83        45

    accuracy                           0.93       752
   macro avg       0.91      0.88      0.90       752
weighted avg       0.93      0.93      0.93       752

Confusion Matrix:
 [[371   0   2   0   0   0]
 [  3  94   2   5   0   0]
 [ 14   1  81   0   0   4]
 [  1   3   2  71   1   2]
 [  4   0   0   1  44   1]
 [  1   0   5   1   1  37]]


# 새로운 음원 파일을 입력받아 학습된 머신러닝 모델을 사용하여 해당 음원의 소음 종류를 분류

In [3]:
# 새로운 음원 파일 분류
def classify_audio(audio_path):
    try:
        # 음원 파일 로드
        y, sr = librosa.load(audio_path, sr=44100)  # 샘플링 레이트(sr)를 44.1kHz로 설정

        # MFCC 특징 추출
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=50)

        # 추출된 MFCC 특징의 평균값을 사용 (각 MFCC 차수별로 하나의 값)
        mfccs_processed = np.mean(mfccs, axis=1)

        # MFCC 특징 벡터를 DataFrame으로 변환
        new_data = pd.DataFrame(mfccs_processed.reshape(1, -1), columns=feature_cols)

        # 데이터 전처리 (스케일링)
        new_data_scaled = scaler.transform(new_data)

        # 예측
        predicted_label_encoded = model.predict(new_data_scaled)
        predicted_label = label_encoder.inverse_transform(predicted_label_encoded)[0]

        return predicted_label

    except Exception as e:
        print(f"Error processing audio file: {e}")
        return "Unknown"


In [4]:
df['label'].value_counts()

기타소음      1865
이륜차경적      519
이륜차주행음     498
차량경적       399
차량사이렌      249
차량주행음      227
Name: label, dtype: int64

## 기타소음 - 고양이

In [5]:
folder_path = "/home/ubuntu/data/etc_noise_data_test_04/동물/15.고양이"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df = pd.DataFrame(results)
results_df.to_csv("cat.csv", index=False)
print("Classification completed. Results saved to 'cat.csv'")

100%|██████████| 274/274 [00:20<00:00, 13.09it/s]

Classification completed. Results saved to 'cat.csv'


## 기타소음 - 항공기

In [6]:
folder_path = "/home/ubuntu/data/etc_noise_data_test_04/교통소음/3.항공기/6.비행기"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df = pd.DataFrame(results)
results_df.to_csv("비행기분류.csv", index=False)
print("Classification completed. Results saved to '비행기분류.csv'")

100%|██████████| 191/191 [00:48<00:00,  3.90it/s]

Classification completed. Results saved to '비행기분류.csv'


## 기타소음 - 헬리콥터

In [7]:
folder_path = "/home/ubuntu/data/etc_noise_data_test_04/교통소음/3.항공기/7.헬리콥터"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df = pd.DataFrame(results)
results_df.to_csv("헬리콥터.csv", index=False)
print("Classification completed. Results saved to '헬리콥터.csv'")

100%|██████████| 435/435 [00:31<00:00, 13.99it/s]

Classification completed. Results saved to '헬리콥터.csv'


## 기타소음 - 가전 청소기

In [8]:
folder_path = "/home/ubuntu/data/etc_noise_data_test_04/가전/12.청소기"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df = pd.DataFrame(results)
results_df.to_csv("청소기.csv", index=False)
print("Classification completed. Results saved to '청소기.csv'")

100%|██████████| 102/102 [00:34<00:00,  2.95it/s]

Classification completed. Results saved to '청소기.csv'


## 교통소음 - 차량 사이렌

In [9]:
folder_path = "/home/ubuntu/data/raw_data/1.Car/2.siren_of_car"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df = pd.DataFrame(results)
results_df.to_csv("자동차사이렌.csv", index=False)
print("Classification completed. Results saved to '자동차사이렌.csv'")

100%|██████████| 1990/1990 [02:26<00:00, 13.54it/s]

Classification completed. Results saved to '자동차사이렌.csv'


In [10]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df)  # 전체 파일 개수
etc_noise_count = results_df[results_df['Predicted Label'] == '기타소음'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'기타소음' Count: {etc_noise_count}")
print(f"'기타소음' Ratio: {etc_noise_ratio:.2f}%")

Total Files: 1990
'기타소음' Count: 163
'기타소음' Ratio: 8.19%


## 교통소음 - 차량 경적

In [11]:
folder_path = "/home/ubuntu/data/raw_data/1.Car/1.horn_of_car"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df = pd.DataFrame(results)
results_df.to_csv("차량경적.csv", index=False)
print("Classification completed. Results saved to '차량경적.csv'")

100%|██████████| 3189/3189 [03:18<00:00, 16.03it/s]

Classification completed. Results saved to '차량경적.csv'


In [12]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df)  # 전체 파일 개수
etc_noise_count = results_df[results_df['Predicted Label'] == '기타소음'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'기타소음' Count: {etc_noise_count}")
print(f"'기타소음' Ratio: {etc_noise_ratio:.2f}%")

Total Files: 3189
'기타소음' Count: 99
'기타소음' Ratio: 3.10%


# CNN

In [ ]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [14]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [15]:
# 1. 학습 데이터 준비
df = pd.read_csv("combined_test_data_200.csv", encoding='utf-8')
feature_cols = [f'mfcc_{i}' for i in range(1, 51)]
df = df[feature_cols + ['category_03']]

# 소음 카테고리 변환 함수
def categorize_noise(category):
    if category in ['이륜차경적']:
        return '이륜차경적'
    elif category in ['이륜차주행음']:
        return '이륜차주행음'
    elif category in ['차량사이렌']:
        return '차량사이렌'
    elif category in ['차량주행음']:
        return '차량주행음'
    elif category in ['차량경적']:
        return '차량경적'
    else:
        return '기타소음'

df['label'] = df['category_03'].apply(categorize_noise)
df = df.drop('category_03', axis=1)

X = df[feature_cols].values  # MFCC 특징 값
y = df['label'].values       # 레이블

# 레이블 인코딩 (문자 → 숫자)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)  # One-hot 인코딩

# 데이터셋 분할 (훈련:테스트 = 80:20)
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42, stratify=y)

# CNN 입력 형태로 변환 (3D 텐서: 샘플 수 x 시간 축 x 특징 수)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# 2. CNN 모델 정의
model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    
    Conv1D(128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(y_categorical.shape[1], activation='softmax')  # 출력층 (카테고리 수만큼 출력)
])

# 3. 모델 컴파일 및 학습
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

# 테스트 정확도 출력
test_loss, test_accuracy = model.evaluate(X_test, y_test)

# 테스트 데이터에 대한 예측 수행
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # 예측된 클래스 인덱스
y_true_classes = np.argmax(y_test, axis=1)  # 실제 클래스 인덱스

# 정확도 계산
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f"Accuracy: {accuracy:.4f}")

# 분류 보고서 출력
class_names = label_encoder.classes_
report = classification_report(y_true_classes, y_pred_classes, target_names=class_names)
print("Classification Report:\n", report)

# 혼동 행렬 출력
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

# 4. 새로운 음원 파일 분류 함수
def classify_audio(audio_path):
    try:
        # 음원 파일 로드
        y, sr = librosa.load(audio_path, sr=44100)

        # MFCC 특징 추출
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=50)

        # 추출된 MFCC 특징의 평균값을 사용 (각 MFCC 차수별로 하나의 값)
        mfccs_processed = np.mean(mfccs, axis=1)

        # CNN 입력 형태로 변환 (3D 텐서)
        new_data = mfccs_processed.reshape(1, -1, 1)

        # 예측 수행
        predicted_label_encoded = np.argmax(model.predict(new_data), axis=-1)[0]
        predicted_label = label_encoder.inverse_transform([predicted_label_encoded])[0]

        return predicted_label

    except Exception as e:
        print(f"Error processing audio file: {e}")
        return "Unknown"


Epoch 1/30
94/94 [==============================] - 1s 8ms/step - loss: 1.7257 - accuracy: 0.5830 - val_loss: 0.7131 - val_accuracy: 0.7779
Epoch 2/30
94/94 [==============================] - 1s 7ms/step - loss: 0.8185 - accuracy: 0.7215 - val_loss: 0.5563 - val_accuracy: 0.8285
Epoch 3/30
94/94 [==============================] - 1s 6ms/step - loss: 0.6448 - accuracy: 0.7740 - val_loss: 0.4467 - val_accuracy: 0.8524
Epoch 4/30
94/94 [==============================] - 1s 6ms/step - loss: 0.5752 - accuracy: 0.8007 - val_loss: 0.3940 - val_accuracy: 0.8697
Epoch 5/30
94/94 [==============================] - 1s 6ms/step - loss: 0.5210 - accuracy: 0.8153 - val_loss: 0.3818 - val_accuracy: 0.8590
Epoch 6/30
94/94 [==============================] - 1s 6ms/step - loss: 0.4750 - accuracy: 0.8306 - val_loss: 0.3300 - val_accuracy: 0.8936
Epoch 7/30
94/94 [==============================] - 1s 6ms/step - loss: 0.4509 - accuracy: 0.8423 - val_loss: 0.3430 - val_accuracy: 0.8936
Epoch 8/30
94/94 [==

## 기타소음 - 고양이

In [16]:
folder_path = "/home/ubuntu/data/etc_noise_data_test_04/동물/15.고양이"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 결과를 DataFrame으로 변환하고 CSV로 저장
results_df = pd.DataFrame(results)
results_df.to_csv("cat_CNN.csv", index=False)
print("Classification completed. Results saved to 'cat_CNN.csv'")


100%|██████████| 274/274 [00:31<00:00,  8.78it/s]

Classification completed. Results saved to 'cat_CNN.csv'


## 기타소음 - 항공기

In [17]:
folder_path = "/home/ubuntu/data/etc_noise_data_test_04/교통소음/3.항공기/6.비행기"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 결과를 DataFrame으로 변환하고 CSV로 저장
results_df = pd.DataFrame(results)
results_df.to_csv("비행기분류_CNN.csv", index=False)
print("Classification completed. Results saved to '비행기분류_CNN.csv'")

100%|██████████| 191/191 [00:52<00:00,  3.66it/s]

Classification completed. Results saved to '비행기분류_CNN.csv'


## 교통소음 - 차량 사이렌

In [18]:
folder_path = "/home/ubuntu/data/raw_data/1.Car/2.siren_of_car"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 결과를 DataFrame으로 변환하고 CSV로 저장
results_df = pd.DataFrame(results)
results_df.to_csv("자동차사이렌_CNN.csv", index=False)
print("Classification completed. Results saved to '자동차사이렌_CNN.csv'")

100%|██████████| 1990/1990 [03:51<00:00,  8.60it/s]

Classification completed. Results saved to '자동차사이렌_CNN.csv'


In [19]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df)  # 전체 파일 개수
etc_noise_count = results_df[results_df['Predicted Label'] == '기타소음'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'기타소음' Count: {etc_noise_count}")
print(f"'기타소음' Ratio: {etc_noise_ratio:.2f}%")

Total Files: 1990
'기타소음' Count: 62
'기타소음' Ratio: 3.12%


## 교통소음 - 차량 경적

In [20]:
folder_path = "/home/ubuntu/data/raw_data/1.Car/1.horn_of_car"
results = []

for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        predicted_label = classify_audio(file_path)
        results.append({"File": filename, "Predicted Label": predicted_label})

# 5. 결과를 DataFrame으로 변환하고 CSV로 저장
results_df = pd.DataFrame(results)
results_df.to_csv("차량경적_CNN.csv", index=False)
print("Classification completed. Results saved to '차량경적_CNN.csv'")

100%|██████████| 3189/3189 [05:13<00:00, 10.19it/s]

Classification completed. Results saved to '차량경적_CNN.csv'


In [21]:
# '기타소음'의 개수와 비율 계산
total_files = len(results_df)  # 전체 파일 개수
etc_noise_count = results_df[results_df['Predicted Label'] == '기타소음'].shape[0]  # '기타소음'으로 분류된 파일 개수
etc_noise_ratio = (etc_noise_count / total_files) * 100  # 비율 계산

# 결과 출력
print(f"Total Files: {total_files}")
print(f"'기타소음' Count: {etc_noise_count}")
print(f"'기타소음' Ratio: {etc_noise_ratio:.2f}%")

Total Files: 3189
'기타소음' Count: 35
'기타소음' Ratio: 1.10%
